# Exploring U.S. Midterm Elections on Polymarket

This notebook uses **Polymarket's Gamma API** (event/market discovery) and **py-clob-client** (prices, order books) to explore prediction market data for the 2026 U.S. midterm elections.

**Architecture:**
- **Gamma API** (`gamma-api.polymarket.com`) — browse events, markets, get slugs and token IDs. Plain HTTP, no auth.
- **CLOB API** (`clob.polymarket.com`) — midpoints, prices, order books by token ID. Uses `py-clob-client`, no auth for read-only.

Each market has **two token IDs**: index `[0]` = **Yes**, index `[1]` = **No**.

> **Install:** `pip install py-clob-client requests pandas matplotlib`

## 1 — Setup

In [2]:
# !pip install py-clob-client requests pandas matplotlib

import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import BookParams

# --- Clients ---
GAMMA = "https://gamma-api.polymarket.com"
clob = ClobClient("https://clob.polymarket.com")  # read-only, no auth

def gamma_get(path, params=None):
    """Helper for Gamma API GET requests."""
    r = requests.get(f"{GAMMA}{path}", params=params)
    r.raise_for_status()
    return r.json()

def pprint(obj, max_len=3000):
    s = json.dumps(obj, indent=2, default=str)
    print(s[:max_len] + ("\n..." if len(s) > max_len else ""))

# Quick health check
print("CLOB:", clob.get_ok())
print("CLOB time:", clob.get_server_time())
print("Gamma:", gamma_get("/events", {"limit": 1})[0]["title"] if gamma_get("/events", {"limit": 1}) else "empty")
print("\n✓ Both APIs connected")

CLOB: OK
CLOB time: 1771819234
Gamma: NBA: Will the Mavericks beat the Grizzlies by more than 5.5 points in their December 4 matchup?

✓ Both APIs connected


## 2 — Fetch an Event by URL Slug

Any `polymarket.com/event/<slug>` URL maps directly to the Gamma API.

In [3]:
def event_from_url(url):
    """Extract slug from a Polymarket URL and fetch the event."""
    slug = url.rstrip("/").split("/")[-1]
    return gamma_get(f"/events/slug/{slug}")

# --- The 2026 House election ---
house_event = event_from_url("https://polymarket.com/event/which-party-will-win-the-house-in-2026")

print(f"Title:       {house_event['title']}")
print(f"Description: {house_event.get('description', 'N/A')[:300]}")
print(f"Active:      {house_event.get('active')}")
print(f"Volume:      ${float(house_event.get('volume', 0) or 0):,.0f}")
print(f"Liquidity:   ${float(house_event.get('liquidity', 0) or 0):,.0f}")
print(f"\nMarkets ({len(house_event.get('markets', []))}):\n")

for m in house_event["markets"]:
    title = m.get("groupItemTitle") or m.get("question", "?")
    tokens = m.get("clobTokenIds", [])
    print(f"  {title:<25}  tokens={tokens}")

Title:       Which party will win the House in 2026?
Description: This market will resolve according to the party that controls the House of Representatives following the 2026 U.S. House elections scheduled for November 3, 2026.

House control is defined as having more than half of the voting members of the U.S. House of Representatives.

If the outcome of this el
Active:      True
Volume:      $3,239,809
Liquidity:   $550,285

Markets (9):

  Other                      tokens=["32806512678351792960664166512761758085909980077229801614845545250268642481454", "24471453313268347694105597233444552965115521151973687481592878562893509526097"]
  Democratic Party           tokens=["83247781037352156539108067944461291821683755894607244160607042790356561625563", "33156410999665902694791064431724433042010245771106314074312009703157423879038"]
  Republican Party           tokens=["65139230827417363158752884968303867495725894165574887635816574090175320800482", "1737121711886212578243807458516621055

## 3 — Inspect CLOB Return Types

Before building analysis, let's see exactly what `get_midpoint` and `get_price` return.

In [4]:
# Pick the first market's Yes token for inspection
sample_market = house_event["markets"][0]
sample_token = sample_market["clobTokenIds"][0]  # Yes token
sample_name = sample_market.get("groupItemTitle") or sample_market.get("question")

print(f"Inspecting: {sample_name}")
print(f"Yes token:  {sample_token}\n")

mid_raw = clob.get_midpoint(sample_token)
price_raw = clob.get_price(sample_token, side="BUY")

print(f"get_midpoint() -> type={type(mid_raw).__name__}")
print(f"  value: {mid_raw}")

print(f"\nget_price(side='BUY') -> type={type(price_raw).__name__}")
print(f"  value: {price_raw}")

book_raw = clob.get_order_book(sample_token)
print(f"\nget_order_book() -> type={type(book_raw).__name__}")
print(f"  attrs: {[a for a in dir(book_raw) if not a.startswith('_')]}")
if book_raw.bids:
    print(f"  sample bid: {book_raw.bids[0]}, type={type(book_raw.bids[0]).__name__}")
    print(f"  bid attrs:  {[a for a in dir(book_raw.bids[0]) if not a.startswith('_')]}")

Inspecting: Other
Yes token:  [



PolyApiException: PolyApiException[status_code=404, error_message={'error': 'No orderbook exists for the requested token id'}]

In [ ]:
# ============================================================
# ADAPT the parse functions based on the output of cell above.
#
# The auto-parser below handles the common return shapes:
#   str "0.62"  |  dict {"mid": "0.62"}  |  float 0.62
# ============================================================

def parse_numeric(raw, preferred_keys=("mid", "price", "midpoint", "value", "px")):
    """Extract a float from whatever the CLOB returns."""
    if isinstance(raw, (int, float)):
        return float(raw)
    if isinstance(raw, str):
        return float(raw)
    if isinstance(raw, dict):
        for key in preferred_keys:
            if key in raw:
                return float(raw[key])
        # fallback: first value that looks numeric
        for v in raw.values():
            try:
                return float(v)
            except (TypeError, ValueError):
                continue
    raise ValueError(f"Cannot parse numeric from {type(raw).__name__}: {raw}")

# Test it
mid_val = parse_numeric(mid_raw)
price_val = parse_numeric(price_raw)
print(f"Parsed midpoint : {mid_val}  ({mid_val:.1%})")
print(f"Parsed buy price: {price_val}")

## 4 — Implied Probabilities: 2026 House Election

The Yes-token midpoint ≈ implied probability of that outcome.

In [ ]:
def get_event_probabilities(event):
    """Fetch midpoints for all markets in a Gamma event."""
    rows = []
    for m in event.get("markets", []):
        title = m.get("groupItemTitle") or m.get("question", "?")
        tokens = m.get("clobTokenIds", [])
        if not tokens:
            continue
        yes_token = tokens[0]
        try:
            mid_val = parse_numeric(clob.get_midpoint(yes_token))
            price_val = parse_numeric(clob.get_price(yes_token, side="BUY"))
            rows.append({
                "Outcome": title,
                "Implied %": round(mid_val * 100, 2),
                "Buy $": round(price_val, 4),
                "Mid $": round(mid_val, 4),
                "Yes Token": yes_token[:20] + "...",
            })
        except Exception as e:
            rows.append({"Outcome": title, "Error": str(e)})
    return pd.DataFrame(rows)

df_house = get_event_probabilities(house_event)
df_house.sort_values("Implied %", ascending=False).reset_index(drop=True)

In [ ]:
# --- Visualization ---
df_plot = df_house.dropna(subset=["Implied %"]).copy()
df_plot = df_plot[df_plot["Implied %"] > 0.5].sort_values("Implied %")

color_map = {"Republican Party": "#E81B23", "Democratic Party": "#0015BC"}
colors = [color_map.get(name, "#888888") for name in df_plot["Outcome"]]

fig, ax = plt.subplots(figsize=(9, max(3, len(df_plot) * 0.7)))
bars = ax.barh(df_plot["Outcome"], df_plot["Implied %"], color=colors, edgecolor="white")
ax.set_xlabel("Implied Probability (%)")
ax.set_title("Which Party Will Win the House in 2026?\n(Polymarket implied probabilities)")
ax.axvline(50, color="gray", linestyle="--", alpha=0.4)
for bar, val in zip(bars, df_plot["Implied %"]):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height() / 2,
            f"{val:.1f}%", va="center", fontsize=10)
plt.tight_layout()
plt.savefig("house_2026_probabilities.png", dpi=150, bbox_inches="tight")
plt.show()

## 5 — Order Book Depth

Thin order books mean a single whale trade can move the price significantly.

In [ ]:
def plot_order_book(token_id, title=""):
    """Fetch and plot cumulative order book depth."""
    book = clob.get_order_book(token_id)

    # book.bids / book.asks are lists of OrderBookSummary objects.
    # Access .price and .size — adjust if your version uses different attrs.
    bids_raw = book.bids or []
    asks_raw = book.asks or []

    # Try .price/.size first, fall back to dict access
    def get_px(lvl):
        return float(getattr(lvl, 'price', None) or lvl.get('price') or lvl.get('px'))
    def get_sz(lvl):
        return float(getattr(lvl, 'size', None) or lvl.get('size') or lvl.get('qty'))

    bids = sorted(bids_raw, key=lambda x: get_px(x), reverse=True)
    asks = sorted(asks_raw, key=lambda x: get_px(x))

    bid_px, bid_cum = [], []
    cum = 0
    for lvl in bids:
        cum += get_sz(lvl)
        bid_px.append(get_px(lvl))
        bid_cum.append(cum)

    ask_px, ask_cum = [], []
    cum = 0
    for lvl in asks:
        cum += get_sz(lvl)
        ask_px.append(get_px(lvl))
        ask_cum.append(cum)

    fig, ax = plt.subplots(figsize=(10, 5))
    if bid_px:
        ax.fill_between(bid_px, bid_cum, alpha=0.3, color="green", step="post")
        ax.step(bid_px, bid_cum, color="green", where="post", label=f"Bids ({len(bids)} lvls, {bid_cum[-1]:,.0f} shares)")
    if ask_px:
        ax.fill_between(ask_px, ask_cum, alpha=0.3, color="red", step="post")
        ax.step(ask_px, ask_cum, color="red", where="post", label=f"Asks ({len(asks)} lvls, {ask_cum[-1]:,.0f} shares)")
    ax.set_xlabel("Price ($)")
    ax.set_ylabel("Cumulative Shares")
    ax.set_title(f"Order Book — {title}" if title else "Order Book")
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot order books for outcomes with > 1% probability
for m in house_event["markets"]:
    title = m.get("groupItemTitle") or m.get("question", "?")
    tokens = m.get("clobTokenIds", [])
    if not tokens:
        continue
    try:
        mid_val = parse_numeric(clob.get_midpoint(tokens[0]))
        if mid_val > 0.01:
            print(f"\n{'='*60}")
            print(f"{title}  (midpoint: {mid_val:.1%})")
            print(f"{'='*60}")
            plot_order_book(tokens[0], title=f"{title} — YES")
    except Exception as e:
        print(f"  Skipped {title}: {e}")

## 6 — Spread & Liquidity Report

Wide spreads = unreliable signal. Thin books = easily manipulated.

In [ ]:
def get_px(lvl):
    return float(getattr(lvl, 'price', None) or lvl.get('price') or lvl.get('px'))
def get_sz(lvl):
    return float(getattr(lvl, 'size', None) or lvl.get('size') or lvl.get('qty'))

def liquidity_report(event):
    rows = []
    for m in event.get("markets", []):
        title = m.get("groupItemTitle") or m.get("question", "?")
        tokens = m.get("clobTokenIds", [])
        if not tokens:
            continue
        try:
            book = clob.get_order_book(tokens[0])
            bids = book.bids or []
            asks = book.asks or []
            best_bid = get_px(bids[0]) if bids else 0
            best_ask = get_px(asks[0]) if asks else 0
            spread = (best_ask - best_bid) if best_bid and best_ask else None
            bid_depth = sum(get_sz(b) for b in bids)
            ask_depth = sum(get_sz(a) for a in asks)

            if spread and spread > 0.10:
                quality = "⚠️ Wide spread"
            elif bid_depth + ask_depth < 100:
                quality = "⚠️ Thin book"
            else:
                quality = "✅ OK"

            rows.append({
                "Outcome": title,
                "Best Bid": f"${best_bid:.3f}",
                "Best Ask": f"${best_ask:.3f}",
                "Spread": f"${spread:.4f}" if spread else "-",
                "Bid Depth": f"{bid_depth:,.0f}",
                "Ask Depth": f"{ask_depth:,.0f}",
                "# Bids": len(bids),
                "# Asks": len(asks),
                "Quality": quality,
            })
        except Exception as e:
            rows.append({"Outcome": title, "Quality": f"❌ {e}"})
    return pd.DataFrame(rows)

df_liq = liquidity_report(house_event)
print(f"Liquidity Report: {house_event['title']}\n")
df_liq

## 7 — Search & Discover More Election Markets

In [ ]:
# Fetch lots of active events and filter for election keywords
election_keywords = {"election", "senate", "house", "congress", "midterm",
                     "governor", "republican", "democrat", "2026"}

election_events = []
offset = 0
PAGE = 100

while offset < 500:
    page = gamma_get("/events", {"limit": PAGE, "offset": offset, "active": True, "closed": False})
    if not page:
        break
    for e in page:
        text = (e.get("title", "") + " " + e.get("description", "")).lower()
        if any(kw in text for kw in election_keywords):
            election_events.append(e)
    offset += PAGE
    if len(page) < PAGE:
        break

print(f"Found {len(election_events)} active election-related events\n")
for e in election_events[:20]:
    n_markets = len(e.get("markets", []))
    print(f"  🗳  {e['title']:<55}  ({n_markets} mkts)  slug={e['slug']}")

In [ ]:
# Load specific events by URL
ELECTION_URLS = [
    "https://polymarket.com/event/which-party-will-win-the-house-in-2026",
    # Add more as you find them, e.g.:
    # "https://polymarket.com/event/which-party-will-win-the-senate-in-2026",
    # "https://polymarket.com/event/some-governor-race",
]

all_events = []
for url in ELECTION_URLS:
    try:
        evt = event_from_url(url)
        all_events.append(evt)
        print(f"✓ {evt['title']}  ({len(evt.get('markets',[]))} markets)")
    except Exception as e:
        print(f"✗ {url}: {e}")

## 8 — Compare Multiple Events

In [ ]:
all_rows = []
for evt in all_events:
    for m in evt.get("markets", []):
        title = m.get("groupItemTitle") or m.get("question", "?")
        tokens = m.get("clobTokenIds", [])
        if not tokens:
            continue
        try:
            mid_val = parse_numeric(clob.get_midpoint(tokens[0]))
            all_rows.append({
                "Event": evt["title"],
                "Outcome": title,
                "Implied %": round(mid_val * 100, 2),
                "Volume": float(m.get("volume", 0) or 0),
            })
        except:
            pass

df_all = pd.DataFrame(all_rows)
if not df_all.empty:
    display(df_all[df_all["Implied %"] > 0.5]
            .sort_values(["Event", "Implied %"], ascending=[True, False])
            .reset_index(drop=True))
else:
    print("Add more URLs to ELECTION_URLS above.")

## 9 — Batch Order Books

In [ ]:
# Batch fetch order books for all Yes tokens in the House event
token_labels = {}
book_params = []
for m in house_event["markets"]:
    title = m.get("groupItemTitle") or m.get("question", "?")
    tokens = m.get("clobTokenIds", [])
    if tokens:
        tid = tokens[0]
        book_params.append(BookParams(token_id=tid))
        token_labels[tid] = title

books = clob.get_order_books(book_params)
print(f"Fetched {len(books)} order books in one call\n")

for book in books:
    label = token_labels.get(book.asset_id, book.asset_id[:20])
    n_bids = len(book.bids) if book.bids else 0
    n_asks = len(book.asks) if book.asks else 0
    print(f"  {label:<30}  bids={n_bids:>3}  asks={n_asks:>3}")

## 10 — Export

In [ ]:
import os
os.makedirs("polymarket_export", exist_ok=True)

exports = {
    "house_probabilities.csv": df_house if 'df_house' in dir() else None,
    "liquidity_report.csv": df_liq if 'df_liq' in dir() else None,
    "all_election_markets.csv": df_all if 'df_all' in dir() and not df_all.empty else None,
}

for fname, df in exports.items():
    if df is not None and not df.empty:
        path = f"polymarket_export/{fname}"
        df.to_csv(path, index=False)
        print(f"✓ {path} ({len(df)} rows)")

print("\nDone!")

## Reading These Numbers Critically

1. **Price ≠ poll.** Capital-weighted, not one-person-one-vote.
2. **Hedging looks like belief.** The CLOB can't distinguish insurance bets from genuine predictions.
3. **Whales move thin markets.** Check the spread & depth (Section 6) before trusting a price.
4. **The platform curates what's bet-able.** Only Polymarket-approved questions become contracts.

---

## Quick Reference

| Task | Code |
|------|------|
| **Gamma: Event from URL** | `gamma_get(f"/events/slug/{slug}")` |
| **Gamma: List events** | `gamma_get("/events", {"limit": 50, "active": True})` |
| **Gamma: List markets** | `gamma_get("/markets", {"limit": 50})` |
| **CLOB: Midpoint** | `clob.get_midpoint(token_id)` |
| **CLOB: Price** | `clob.get_price(token_id, side="BUY")` |
| **CLOB: Order book** | `clob.get_order_book(token_id)` |
| **CLOB: Batch books** | `clob.get_order_books([BookParams(token_id=t) for t in ids])` |
| **CLOB: Last trade** | `clob.get_last_trade_price(token_id)` |
| **CLOB: Health** | `clob.get_ok()`, `clob.get_server_time()` |